# Introduction to Tensors

- Simply a number, vector, matrix or any n-dim array.

- All elements inside a tensor must be of the same data type, and do to so, python can perform implicit data     type conversion.

In [1]:
import torch

In [2]:
# Number

t_num = torch.tensor(4.)

print(t_num)
print(t_num.dtype)
print(t_num.shape)

tensor(4.)
torch.float32
torch.Size([])


In [3]:
# Vector

t_vec = torch.tensor([
    [1., 2, 3, 4]
])

print(t_vec)
print(t_vec.dtype)
print(t_vec.shape)

tensor([[1., 2., 3., 4.]])
torch.float32
torch.Size([1, 4])


In [4]:
# Matrix

t_mat = torch.tensor([
    [5, 6],
    [7, 8],
    [9, 10]
])

print(t_mat)
print(t_mat.dtype)
print(t_mat.shape)

tensor([[ 5,  6],
        [ 7,  8],
        [ 9, 10]])
torch.int64
torch.Size([3, 2])


In [5]:
# 3D Array. Each row and column should have same number of elements as every other row and column respectively.

t_3d = torch.tensor([
    [[1, 2, 3], [4, 5, 6]],
    [[11, 12, 13], [14, 15, 16]]
])

print(t_3d)
print(t_3d.dtype)
print(t_3d.shape)

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[11, 12, 13],
         [14, 15, 16]]])
torch.int64
torch.Size([2, 2, 3])


## Tensor Operations and Gradients

- Requires grad enables calculation of derivative with respect to the tensor for which that property is set as   True.

In [6]:
# Performing operations

m = torch.tensor(3.)
x = torch.tensor(4., requires_grad=True)
b = torch.tensor(8., requires_grad=True)

y = m * x + b
print(y)
print(y.shape)

tensor(20., grad_fn=<AddBackward0>)
torch.Size([])


In [7]:
# Computing derivatives

y.backward()

In [8]:
print(f'dy/dx = {x.grad}')
print(f'dy/dm = {m.grad}')
print(f'dy/db = {b.grad}')

dy/dx = 3.0
dy/dm = None
dy/db = 1.0


## NumPy with PyTorch

In [9]:
import numpy as np

In [10]:
np_arr = np.array([
    [1, 2],
    [3, 4]
])

np_arr

array([[1, 2],
       [3, 4]])

In [11]:
# NumPy array --> Tensor

y = torch.from_numpy(np_arr)
y

tensor([[1, 2],
        [3, 4]])

In [12]:
# Tensor --> NumPy array

z = y.numpy()
z

array([[1, 2],
       [3, 4]])

# Linear Regression with PyTorch

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

In [13]:
# Input (temp, rainfall, humidity)

inputs = np.array([
    [73, 67, 43], 
    [91, 88, 64], 
    [87, 134, 58], 
    [102, 43, 37], 
    [69, 96, 70],
], dtype='float32')

inputs.shape

(5, 3)

In [14]:
# Targets (apples, oranges)

targets = np.array([
    [56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]
], dtype='float32')

targets.shape

(5, 2)

In [15]:
# Convert inputs and targets to tensors

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs.dtype)
print(targets.dtype)

torch.float32
torch.float32


In [16]:
# Random weights and biases for initialization

w = torch.randn(2, 3, requires_grad=True) # Dimension = (2,3)
b = torch.randn(2, requires_grad=True)

print(w.shape, w.dtype)
print(b.shape, b.dtype)

torch.Size([2, 3]) torch.float32
torch.Size([2]) torch.float32


In [17]:
'''
@: matrix multiplication

shape of w: (2, 3)
shape of w.T: (3, 2)
shape of x: (5, 3)
shape of b: (2)

x @ w.t() ---> output of shape (5, 2)

Since b's shape is (2), PyTorch will broadcast its values when performing addition,
thus making its shape (5, 2)'''

def model(x):
    return x @ w.t() + b

In [18]:
predictions = model(inputs)
predictions

tensor([[ 39.9710, 195.4098],
        [ 55.2833, 235.3807],
        [ -3.3652, 340.9050],
        [ 87.1394, 201.2134],
        [ 31.7867, 205.8769]], grad_fn=<AddBackward0>)

In [19]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [20]:
# Getting numpy arrays from tensors

true_y = targets.detach().numpy()

In [21]:
# Getting numpy arrays from tensors

predicted_y = predictions.detach().numpy()

In [22]:
# Compute loss

from sklearn.metrics import mean_squared_error

RMSE = np.sqrt(mean_squared_error(true_y, predicted_y))
RMSE

116.93157

In [23]:
# Computing loss with UD function

def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [24]:
# Compute loss

loss = mse(predictions, targets)
print(loss)

tensor(13672.9922, grad_fn=<DivBackward0>)


## Computing Gradients

In [25]:
loss.backward()

In [26]:
# Gradients for weights

print(w)
print('-----------------')
print(w.grad)

tensor([[ 0.9484, -1.0646,  0.9808],
        [ 1.6707,  2.2122, -1.7409]], requires_grad=True)
-----------------
tensor([[-2485.1240, -4753.8887, -2401.4153],
        [12443.1143, 12697.7188,  7641.7515]])


In [27]:
# To flush out previous gradient values

w.grad.zero_()
b.grad.zero_()

print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Training model using Gradient Descent

In [28]:
# Generate predictions
predictions = model(inputs)
print(predictions)

tensor([[ 39.9710, 195.4098],
        [ 55.2833, 235.3807],
        [ -3.3652, 340.9050],
        [ 87.1394, 201.2134],
        [ 31.7867, 205.8769]], grad_fn=<AddBackward0>)


In [29]:
# Calculate the loss
loss = mse(predictions, targets)
print(loss)

tensor(13672.9922, grad_fn=<DivBackward0>)


In [30]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-2485.1240, -4753.8887, -2401.4153],
        [12443.1143, 12697.7188,  7641.7515]])
tensor([-34.0370, 143.7572])


In [31]:
# Adjust weights & reset gradients

with torch.no_grad(): # Don't track values for gradient related work
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [32]:
print(w)
print(b)

tensor([[ 0.9733, -1.0170,  1.0048],
        [ 1.5463,  2.0852, -1.8173]], requires_grad=True)
tensor([-0.1109,  0.0851], requires_grad=True)


In [35]:
# Model has lesser loss with gradient descent

predictions = model(inputs)
loss = mse(predictions, targets)
print(loss)

tensor(9946.9951, grad_fn=<DivBackward0>)
